# Preprocessing the Datasets


## Importing the Libraries

In [8]:
import pandas as pd
import os


In [24]:
# Load the uploaded TSV files
extot_path = "Datasets/Raw/estat_tour_dem_extot.tsv"
tnage_path = "Datasets/Raw/estat_tour_dem_tnage.tsv"
ttsex_path = "Datasets/Raw/estat_tour_dem_ttsex.tsv"

# Load first few rows of each dataset
extot = pd.read_csv(extot_path, sep="\t")
tnage = pd.read_csv(tnage_path, sep="\t")
ttsex = pd.read_csv(ttsex_path, sep="\t")


In [33]:
extot_head = extot.head()
extot_cols = extot.columns.tolist()


extot_head, extot_cols

(  freq,purpose,duration,c_dest,expend,statinfo,unit,geo\TIME_PERIOD   2012   \
 0                A,PER,N1-3,DOM,ACCOM,AVG_NGT,EUR,AL                     :    
 1                A,PER,N1-3,DOM,ACCOM,AVG_NGT,EUR,AT                 66.02    
 2                A,PER,N1-3,DOM,ACCOM,AVG_NGT,EUR,BE                 36.76    
 3                A,PER,N1-3,DOM,ACCOM,AVG_NGT,EUR,BG                 4.58 b   
 4                A,PER,N1-3,DOM,ACCOM,AVG_NGT,EUR,CH                 39.76    
 
     2013    2014    2015    2016     2017    2018    2019    2020    2021   \
 0      :       :       :       :     2.38       :    3.88    9.51       :    
 1  50.64   46.01   51.48   54.07    55.03   56.24   56.22   51.82   50.29    
 2  39.50   37.12   37.13   34.41   39.76 b  34.96   37.88   37.23   40.52    
 3   5.42    6.66    6.40    6.41     5.85    5.60    8.66    9.62   12.74    
 4  47.91   46.39   53.58   52.03    53.80   52.26   55.43   60.78   70.47    
 
     2022    2023   
 0      :       :   


In [31]:
tnage_cols = tnage.columns.tolist()
tnage_head = tnage.head()

tnage_head, tnage_cols


(  freq,purpose,c_dest,duration,age,unit,geo\TIME_PERIOD     2012      2013   \
 0                        A,PER,DOM,N1-3,Y15-24,NR,AL           :         :    
 1                        A,PER,DOM,N1-3,Y15-24,NR,AT     1716783   1881068    
 2                        A,PER,DOM,N1-3,Y15-24,NR,BE     194617 u   287394    
 3                        A,PER,DOM,N1-3,Y15-24,NR,BG     989104 b   881981    
 4                        A,PER,DOM,N1-3,Y15-24,NR,CH     958967 u  799593 u   
 
       2014      2015       2016      2017      2018      2019       2020   \
 0        :         :          :    815878         :    773740     486390    
 1  1711578   1577217    1956490   1837386   1871739   2044466    1543619    
 2   194586    378535     214340   409604 b   520075    535349     513967    
 3   691500    467668     475164    911906    733618    631363     352139    
 4  694026 u  856549 u  1007354 u  666201 u  456367 u  898414 u  1169447 u   
 
        2021       2022      2023   
 0         

In [32]:
ttsex_head = ttsex.head()
ttsex_cols = ttsex.columns.tolist()

ttsex_head, ttsex_cols


(  freq,c_dest,purpose,duration,sex,unit,geo\TIME_PERIOD      2012      2013   \
 0                             A,DOM,PER,N1-3,F,NR,AL            :         :    
 1                             A,DOM,PER,N1-3,F,NR,AT      3155661   3417521    
 2                             A,DOM,PER,N1-3,F,NR,BE       756712    907695    
 3                             A,DOM,PER,N1-3,F,NR,BG     1029215 b   953716    
 4                             A,DOM,PER,N1-3,F,NR,CH      2156668   2350827    
 
       2014      2015      2016       2017      2018      2019      2020   \
 0        :         :         :    1213286         :   1424643    938050    
 1  3616832   3465207   3716126    3505223   3874006   4113956   3406028    
 2   742464   1152356    938572   1118499 b  1281495   1487201   1177284    
 3   931887    703343    832847    1377737   1188827   1101915    610053    
 4  2072561   2415184   1950026    2213164   1973983   2113764   2120157    
 
       2021      2022      2023   
 0        :  

In [34]:
out_dir = "Datasets/Processed"
os.makedirs(out_dir, exist_ok=True)

In [39]:
def preprocess(file_path, meta_cols, id_builder):
    df = pd.read_csv(file_path, sep="\t")
    
    # Fix column names (strip spaces)
    df.columns = df.columns.str.strip()
    
    # Split first column into metadata
    parts = df[df.columns[0]].str.split(",", expand=True)
    df[meta_cols] = parts.iloc[:, :len(meta_cols)]
    
    # Year columns (now clean)
    year_cols = [c for c in df.columns if c.isdigit()]
    
    # Melt into long format
    df_long = df.melt(
        id_vars=meta_cols,
        value_vars=year_cols,
        var_name="year",
        value_name="value"
    )
    
    # Clean value column
    df_long["value"] = df_long["value"].astype(str)
    df_long["value"] = df_long["value"].str.replace(":", "", regex=False)
    df_long["value"] = df_long["value"].str.replace(r"[^\d\.\-]", "", regex=True)
    df_long["value"] = pd.to_numeric(df_long["value"], errors="coerce")
    df_long = df_long.dropna(subset=["value"])
    
    # Build trip_id
    df_long["trip_id"] = df_long.apply(id_builder, axis=1)
    
    return df_long


In [40]:
# Define schemas
extot_cols = ['freq','purpose','duration','c_dest','expend','statinfo','unit','geo']
tnage_cols = ['freq','purpose','c_dest','duration','age','unit','geo']
ttsex_cols = ['freq','c_dest','purpose','duration','sex','unit','geo']


In [41]:
# Define ID builders
extot_id = lambda row: f"{row['year']}_{row['geo']}_{row['purpose']}_{row['duration']}"
tnage_id = lambda row: f"{row['year']}_{row['geo']}_{row['purpose']}_{row['duration']}_{row['age']}"
ttsex_id = lambda row: f"{row['year']}_{row['geo']}_{row['purpose']}_{row['duration']}_{row['sex']}"

In [42]:
# Preprocess
extot_clean = preprocess(extot_path, extot_cols, extot_id)
tnage_clean = preprocess(tnage_path, tnage_cols, tnage_id)
ttsex_clean = preprocess(ttsex_path, ttsex_cols, ttsex_id)

In [43]:
# Save
extot_clean.to_csv(os.path.join(out_dir, "extot_ready.csv"), index=False)
tnage_clean.to_csv(os.path.join(out_dir, "tnage_ready.csv"), index=False)
ttsex_clean.to_csv(os.path.join(out_dir, "ttsex_ready.csv"), index=False)